In [ ]:
import os
import sys
if ".." not in sys.path:
    sys.path.insert(0, "..")

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tixti_utils as tut

In [ ]:
DATA_DIR = '../../datasets/NNN/'
dat = pd.read_pickle(os.path.join(DATA_DIR, ('face_roi_data.pkl')))
print(f'Unique face ROIs: {list(dat['roi'].unique())}')
ROI_LIST = dat['roi'].unique()

all_ks = {}
for _r in ['face', 'body', 'object']:
    subdir = os.path.join(DATA_DIR, f'{_r}_mins.pkl')
    df = pd.read_pickle(subdir)
    all_ks.update(df)

In [ ]:
for _roi in ROI_LIST:


    
    scores = tut.landscape(dat, _roi)
    order = np.argsort(scores)[::-1]

    customp = sns.color_palette('Reds_r', len(scores))
    rxlim = len(scores)
    
    fig, axes = plt.subplots(1, 2, figsize=(20, 3))
    ax = axes[0]
    ax.vlines(range(len(scores)), 0, scores[order], 
              colors=customp, linewidth=0.6, alpha=0.8)  # vertical sticks from 0 to score

    ax.vlines(all_ks[_roi][0], scores.min(), scores.max(), 
              color='black', linestyle='--')

    kth_score = scores[order][all_ks[_roi][0]]
    ax.text(x=200, y=scores.max()*0.75, s=f'k-th score: {kth_score}')
    
    ax.hlines(kth_score, -5, len(scores),
              color='black', linestyle='-', linewidth=0.5)

    ax.set_xlim(left=-1, right=rxlim)
    ax.set_xlabel('')
    ax.set_ylabel('Response (normalized)')
    ax.set_title(_roi)

    ax = axes[1]
    bin_size = 5
    N = 100
    y_top = scores[order][:N]
    n_bins = y_top.size // bin_size
    y_binned = y_top[:n_bins * bin_size].reshape(n_bins, bin_size).mean(axis=1)
    sns.barplot(y_binned, ax=ax)
    ax.set_xticklabels(np.arange(bin_size, N, bin_size))

    sns.despine(trim=True, offset=5, fig=fig)
    plt.show()

    